In [2]:
!pip install tensorflow-gpu==2.0.0-beta

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('./gdrive/My Drive/NLP_Project')
import logging

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json

from generator import DataGenerator
import tensorflow as tf

from RN import RelationalNetwork,objectify
from resnet import ResnetV1_FCNN
from transformer import Encoder,create_padding_mask
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input,Concatenate,Reshape
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks  import ModelCheckpoint

In [5]:
data_dir = 'formatted'
with open(os.path.join(data_dir,'params.json'),'r') as f:
    input_vocab_size = json.load(f)
    input_vocab_size = input_vocab_size['vocab_size']

FileNotFoundError: ignored

In [0]:
#defining model's inputs 
img_shape = (224,224,3) 
imgL = Input(shape=img_shape, name="imgL", dtype = tf.float32)
imgR = Input(shape=img_shape, name="imgR", dtype = tf.float32)
sent = Input(shape=(40,), name="sent", dtype = tf.int32)

#embedding images
fcnn = ResnetV1_FCNN(img_shape,20)
em_imgL = fcnn(imgL)
em_imgR = fcnn(imgR)
em_imgs = Concatenate(2)([em_imgL,em_imgR])

#embedding sentence
print('creating transformer encoder')
enc_mask = create_padding_mask(sent)
encoder = Encoder(num_layers=4, d_model=128, num_heads=8, dff=512, input_vocab_size=input_vocab_size)
em_sent = encoder(sent,training=True,mask=enc_mask)

#getting prediction from the Relational Neural Network 
print('creating relational network')
RN = RelationalNetwork(em_sent.shape[2],em_imgs.shape[2])
pred = RN([em_sent,em_imgs],training=True)

#compile model
print('compiling model')
model = Model(inputs=[imgL,imgR,sent],outputs=pred)
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model_path = 'checkpoints/first_model.{epoch:03d}.h5'
checkpoint = ModelCheckpoint(filepath=model_path,save_best_only=True)
datagen = DataGenerator(data_dir)
callbacks = [checkpoint]

In [0]:
model.fit_generator(datagen, epochs=200, verbose=1, workers=4,callbacks=callbacks)